## Load the packages

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from pathlib import Path
from tqdm.auto import tqdm

/root/miniconda3/envs/rag_demo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define the global constants

Note that this runs completely offline.
1. We use the `Alibaba-NLP/gte-large-en-v1.5` model as the embedding model from HuggingFace.
2. We use the `premai-io/prem-1B-chat` for the chat model instead of OpenAI (GPT 3.5 or GPT 4).

In [2]:
DEVICE = 'cuda'
EMBED_MODEL_NAME = "Alibaba-NLP/gte-large-en-v1.5"
CHAT_MODEL_NAME = 'premai-io/prem-1B-chat'

CHUNK_SIZE = 400
CHUNK_OVERLAP = 40

RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the users's question. If you can't find the answer in the context, just say that you don't know. Keep the answer short and concise.

Context: ```
{context}
```"""

## Embed the docs

#### Load the docs

In [3]:
docs = []

for f in tqdm(list(Path('docs').iterdir())):
    if f.suffix == '.pdf':
        loader = PyPDFLoader(f)
        docs.extend(loader.load())

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:38<00:00,  1.92s/it]


#### Split the docs

In [4]:
tokenizer = AutoTokenizer.from_pretrained(CHAT_MODEL_NAME)
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
splits = text_splitter.split_documents(docs)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Create the vectorstore

In [5]:
model_kwargs = {'device': DEVICE, 'trust_remote_code': True}
encode_kwargs = {'normalize_embeddings': False}
embedding_gen = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_NAME,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_gen)

/root/miniconda3/envs/rag_demo/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


def generate_result(model, tokenizer, messages, terminators):
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Generate
    inputs = tokenizer(prompt, return_attention_mask=False, return_tensors="pt", add_special_tokens=False)
    input_ids = inputs['input_ids']
    input_ids = input_ids.to(model.device)
    res = model.generate(input_ids=input_ids, max_new_tokens=1000, pad_token_id=tokenizer.pad_token_id, eos_token_id=terminators, do_sample=False)
    generated_text = tokenizer.decode(res[0][input_ids.shape[1]:], skip_special_tokens=True).strip()
    return generated_text


def ask_question(ques, vectorstore, model, tokenizer, terminators):
    docs = vectorstore.similarity_search(ques, k=7)
    relevant_context = format_docs(docs)

    prompt = ChatPromptTemplate.from_messages(
        [("system", RAG_SYSTEM_PROMPT), ("user", "{input}")]
    )
    messages = prompt.invoke({'context': relevant_context, 'input': 'hi'}).messages
    messages = [
        {'role': 'system', 'content': messages[0].content},
        {'role': 'user', 'content': messages[1].content},
    ]
    res = generate_result(model, tokenizer, messages, terminators)
    return {'response': res, 'relevant_docs': [d.page_content for d in docs]}

## Chat

#### Load the `premai-io/prem-1B-chat` model from Huggingface.

In [7]:
# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained('premai-io/prem-1B-chat', torch_dtype=torch.bfloat16)
model = model.to(DEVICE)

# Setup terminators
terminators = [tokenizer.eos_token_id, tokenizer.encode('<|eot_id|>', add_special_tokens=False)[0]]

#### Ask the questions

In [49]:
questions = [
    "How does the proposed directive address the challenges related to the application of the Product Liability Directive (PLD) to products in the modern digital economy, specifically in relation to software and AI-enabled products?",
    "What are the proposed changes to the limitations on making compensation claims, such as the threshold for property damage and the period of liability for manufacturers?",
    "How does the NIS 2 Directive (Directive (EU) 2022/2555) ensure a high common level of cybersecurity across the Union, and what are the specific obligations for Member States in terms of national cybersecurity strategies?",
    "How does the Data Act ensure the protection of trade secrets when data holders are required to share data with users or third parties, and what measures can data holders take to preserve the confidentiality of their trade secrets?",
    "What measures are proposed to ease the burden of proof for injured persons in complex cases involving pharmaceuticals, smart products, or AI-enabled products?",
]

In [50]:
for i, ques in enumerate(questions):
    res = ask_question(ques, vectorstore, model, tokenizer, terminators)
    print(f'Question {i+1}: {ques}')
    print()
    print(f"Assistant Response {i+1}: {res['response']}")
    print()
    print('-'*100)
    print()

Question 1: How does the proposed directive address the challenges related to the application of the Product Liability Directive (PLD) to products in the modern digital economy, specifically in relation to software and AI-enabled products?

Assistant Response 1: hi,

the proposal is to modernize rules on machinery, radio equipment and general product safety in the european union. The proposal aims to ensure that when products cause harm, they are liable to compensation. The proposal also aims to ensure that when products cause harm, they are liable to compensation. The proposal is in line with the commission's priorities to make the european union fit for the digital age and to build a future-ready economy that works for people. The proposal complements the digital -by-default modernization process that the commission is undertaking. The proposal is in line with the commission's priorities to make the european union fit for the digital age and to build a future-ready economy that works